In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential

# Get the data

In [ ]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
test_df = pd.read_csv('test.csv')

### Image preprocessing

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 180
IMG_WIDTH = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "training_set/training_set/",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  )

Found 8005 files belonging to 2 classes.
Using 6404 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  "training_set/training_set/",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  )

Found 8005 files belonging to 2 classes.
Using 1601 files for validation.


In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  "test_set/test_set/",
  #validation_split=0.2,
  #subset="test",
  #seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  )

Found 2023 files belonging to 2 classes.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['cats', 'dogs']


For the training process we will use a total of 8005 images:


*   6404 for training
*   1601 for validation



### Data standardisation

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# normalization_layer = layers.Rescaling(1./255)

In [ ]:
# if ESTANDARIZAR_DATOS:
#   normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
#   image_batch, labels_batch = next(iter(normalized_ds))
#   first_image = image_batch[0]
#   # Notice the pixel values are now in `[0,1]`.
#   print(np.min(first_image), np.max(first_image))

# Model

In [ ]:
num_classes = len(class_names)

In [ ]:
custom_model = Sequential()

custom_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = "relu", input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)))
custom_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = "relu"))
custom_model.add(MaxPool2D(pool_size=(2,2)))

custom_model.add(Dropout(0.25))

custom_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = "relu"))
custom_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = "relu"))
custom_model.add(MaxPool2D(pool_size = (2,2)))

custom_model.add(Dropout(0.25))

custom_model.add(Conv2D(filters = 16, kernel_size = (3,3), activation = "relu"))
custom_model.add(Conv2D(filters = 16, kernel_size = (3,3), activation = "relu"))
custom_model.add(MaxPool2D(pool_size = (2,2)))

custom_model.add(Flatten())
custom_model.add(Dense(units = 12, activation = "relu"))
custom_model.add(Dense(units = 1, activation = "sigmoid"))

In [ ]:
# Make sure you have frozen the correct layers
for i, layer in enumerate(custom_model.layers):
    print(i, layer.name, layer.trainable)

0 conv2d True
1 conv2d_1 True
2 max_pooling2d True
3 dropout True
4 conv2d_2 True
5 conv2d_3 True
6 max_pooling2d_1 True
7 dropout_1 True
8 conv2d_4 True
9 conv2d_5 True
10 max_pooling2d_2 True
11 flatten True
12 dense True
13 dense_1 True


In [ ]:
custom_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 176, 176, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 88, 88, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 88, 88, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 86, 86, 32)        18464     
                                                                 
 conv2d_3 (Conv2D)           (None, 84, 84, 32)        9248      
                                                        

In [ ]:
custom_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model_history = custom_model.fit(train_ds, validation_data = val_ds, epochs = 25)

Epoch 1/25
201/201 [==============================] - 48s 119ms/step - loss: 0.7798 - accuracy: 0.4889 - val_loss: 0.6932 - val_accuracy: 0.4884
Epoch 2/25
201/201 [==============================] - 21s 106ms/step - loss: 0.6932 - accuracy: 0.5034 - val_loss: 0.6932 - val_accuracy: 0.4878
Epoch 3/25
201/201 [==============================] - 21s 106ms/step - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.4878
Epoch 4/25
201/201 [==============================] - 21s 106ms/step - loss: 0.6932 - accuracy: 0.5034 - val_loss: 0.6933 - val_accuracy: 0.4878
Epoch 5/25
201/201 [==============================] - 21s 106ms/step - loss: 0.6932 - accuracy: 0.5034 - val_loss: 0.6932 - val_accuracy: 0.4878
Epoch 6/25
201/201 [==============================] - 21s 106ms/step - loss: 0.6932 - accuracy: 0.5034 - val_loss: 0.6933 - val_accuracy: 0.4878
Epoch 7/25
201/201 [==============================] - 21s 107ms/step - loss: 0.6932 - accuracy: 0.5034 - val_loss: 0.6932 - val_ac

KeyboardInterrupt: ignored

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']

loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Model evaluation

In [ ]:
# Generate generalization metrics
score = custom_model.evaluate(test_ds, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')